# 💾 Zapisywanie Modeli (Pickle & Joblib)

Modele Machine Learning to tak naprawdę obiekty w pamięci RAM (zbiory macierzy, wag i reguł if-else).
Kiedy wyłączasz Pythona, one znikają.

Aby użyć modelu na stronie internetowej (np. w Django, Flask, Streamlit), musimy go:
1.  **Wytrenować** (raz).
2.  **Zapisać do pliku** (zserializować).
3.  **Wczytać z pliku** na serwerze.

Mamy do tego dwa główne narzędzia:
*   **Pickle:** Standardowy moduł Pythona.
*   **Joblib:** Działa jak Pickle, ale jest zoptymalizowany dla dużych tablic NumPy (czyli idealny dla Scikit-Learn).

In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# 1. SZYBKI TRENING
# Trenujemy prosty Las Losowy na Irysach
iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("✅ Model wytrenowany.")
print(f"Skuteczność: {model.score(X_test, y_test)*100:.2f}%")

# Sprawdźmy przykładową predykcję dla pierwszego kwiatka z testu
sample_flower = [X_test[0]]
print(f"Przykładowa predykcja dla {sample_flower}: Klasa {model.predict(sample_flower)[0]}")

✅ Model wytrenowany.
Skuteczność: 100.00%
Przykładowa predykcja dla [array([6.1, 2.8, 4.7, 1.2])]: Klasa 1


## Metoda 1: Pickle (Standard)

`pickle` zamienia dowolny obiekt Pythona w strumień bajtów (zer i jedynek), który można zapisać na dysku.

In [2]:
import pickle

# --- ZAPISYWANIE (DUMP) ---
filename = "moj_model_iris.pkl"

# 'wb' oznacza Write Binary (zapisz jako binarne zera i jedynki)
with open(filename, 'wb') as file:
    pickle.dump(model, file)

print(f"💾 Model zapisany do pliku: {filename}")
print("Możesz teraz usunąć zmienną 'model' z pamięci, a plik zostanie na dysku.")

# --- SYMULACJA AWARII (Usuwamy model) ---
del model
try:
    print(model)
except NameError:
    print("❌ Błąd! Zmienna 'model' nie istnieje. Pamięć wyczyszczona.")

# --- WCZYTYWANIE (LOAD) ---
# 'rb' oznacza Read Binary
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

print("\n♻️ Model wczytany z pliku!")
print(f"Czy działa? Sprawdźmy tego samego kwiatka: {sample_flower}")
print(f"Predykcja: Klasa {loaded_model.predict(sample_flower)[0]}")

💾 Model zapisany do pliku: moj_model_iris.pkl
Możesz teraz usunąć zmienną 'model' z pamięci, a plik zostanie na dysku.
❌ Błąd! Zmienna 'model' nie istnieje. Pamięć wyczyszczona.

♻️ Model wczytany z pliku!
Czy działa? Sprawdźmy tego samego kwiatka: [array([6.1, 2.8, 4.7, 1.2])]
Predykcja: Klasa 1


## Metoda 2: Joblib (Zalecane dla ML)

Jeśli Twój model waży 500 MB (bo ma tysiące drzew albo wielkie macierze), `pickle` może być wolny.
`joblib` radzi sobie z tym znacznie szybciej dzięki kompresji i optymalizacji pamięci.

In [3]:
import joblib

# --- ZAPISYWANIE ---
joblib_file = "moj_model_iris.joblib"
joblib.dump(loaded_model, joblib_file)

print(f"💾 Model zapisany przy użyciu Joblib: {joblib_file}")

# --- WCZYTYWANIE ---
model_v2 = joblib.load(joblib_file)

print("♻️ Model wczytany (Joblib)!")
print(f"Weryfikacja skuteczności: {model_v2.score(X_test, y_test)*100:.2f}%")

💾 Model zapisany przy użyciu Joblib: moj_model_iris.joblib
♻️ Model wczytany (Joblib)!
Weryfikacja skuteczności: 100.00%


## Symulacja Produkcji (API)

Wyobraź sobie, że piszesz kod dla backendu (np. FastAPI).
Nie trenujesz modelu w kodzie serwera!
Twój kod serwera wyglądałby mniej więcej tak:

In [4]:
# To jest symulacja małej aplikacji
def predict_flower_type(sepal_len, sepal_wid, petal_len, petal_wid):
    # 1. Ładowanie modelu (normalnie robimy to raz przy starcie serwera)
    # Zauważ: Nie ma tu kodu treningowego! Tylko gotowy plik.
    ai_brain = joblib.load("moj_model_iris.joblib")
    
    # 2. Przygotowanie danych
    data = [[sepal_len, sepal_wid, petal_len, petal_wid]]
    
    # 3. Predykcja
    prediction = ai_brain.predict(data)[0]
    
    # 4. Tłumaczenie na ludzki
    names = {0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'}
    return names[prediction]

# KLIENT WCHODZI NA STRONĘ I WPISUJE DANE:
wynik = predict_flower_type(5.1, 3.5, 1.4, 0.2)
print(f"🤖 AI mówi: To jest {wynik}!")

🤖 AI mówi: To jest Setosa!


## 🧠 Podsumowanie: Nie bierz cukierków od nieznajomych

Zapisywanie modeli jest super, ale ma jeden mroczny sekret.

**Tu jest haczyk (Bezpieczeństwo).**
Pliki `pickle` i `joblib` mogą zawierać **dowolny kod Python**.
Jeśli ściągniesz z internetu plik `super_model.pkl` od nieznanego autora i zrobisz `pickle.load()`, ten plik może w trakcie ładowania:
1.  Skasować Twój dysk.
2.  Ukraść hasła.
3.  Zainstalować wirusa.

**Zasada AI Engineera:**
Nigdy nie ładuj modeli (`.pkl`, `.joblib`) z niezaufanych źródeł.
W bezpiecznym środowisku produkcyjnym coraz częściej używa się formatu **ONNX** (Open Neural Network Exchange), który zapisuje samą matematykę (wagi), a nie kod wykonywalny. Ale na początek Joblib wystarczy.